In [1]:
!pip install --upgrade keras

In [ ]:
import os
import tensorflow as tf
import tensorflow_hub as hub
from google.colab import drive

# Monter Google Drive
drive.mount('/content/drive')

# Chemins des données
data_dir = "/content/drive/MyDrive/food/images"  # Dossier contenant les images
meta_dir = "/content/drive/MyDrive/food/meta"    # Dossier contenant les fichiers meta
train_file = os.path.join(meta_dir, "train.txt")
test_file = os.path.join(meta_dir, "test.txt")

# Vérification des dossiers et fichiers
if os.path.exists(data_dir):
    print("Le dossier des images existe :", data_dir)
else:
    print("Le dossier des images n'existe pas :", data_dir)

if os.path.exists(meta_dir):
    print("Le dossier meta existe :", meta_dir)
else:
    print("Le dossier meta n'existe pas :", meta_dir)

if os.path.exists(train_file):
    print("Le fichier train.txt existe :", train_file)
else:
    print("Le fichier train.txt n'existe pas :", train_file)

if os.path.exists(test_file):
    print("Le fichier test.txt existe :", test_file)
else:
    print("Le fichier test.txt n'existe pas :", test_file)

# Charger les classes disponibles dans 'classes.txt'
with open(os.path.join(meta_dir, "classes.txt"), "r") as f:
    all_classes = [line.strip() for line in f.readlines()]

# Classes que nous voulons utiliser
target_classes = ['steak', 'tacos', 'ice_cream', 'hamburger', 'hot_dog']
# Vérifier les classes disponibles parmi les classes cibles
available_classes = [cls for cls in all_classes if cls in target_classes]
num_classes = len(available_classes)

# Créer une table de correspondance entre les labels et leurs indices
keys_tensor = tf.constant(available_classes)
values_tensor = tf.range(len(available_classes), dtype=tf.int32)
table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(keys_tensor, values_tensor),
    default_value=-1  # Valeur par défaut si une classe n'est pas trouvée
)

# Fonction de prétraitement des images
image_size = (224, 224)  # Taille de l'image pour le modèle

def preprocess_image(image_path, label):
    # Charger et décoder l'image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, image_size)
    image = tf.cast(image, tf.float32) / 255.0  # Normalisation des pixels à [0, 1]

    # Mapper le label à son index
    label_idx = table.lookup(label)
    return image, label_idx

# Fonction pour charger les données à partir des fichiers train.txt et test.txt
def load_data(file_path):
    images = []
    labels = []
    with open(file_path, "r") as f:
        for line in f:
            label, image_name = line.strip().split("/")  # Format attendu : label/nom_image
            if label in available_classes:
                image_path = os.path.join(data_dir, label, f"{image_name}.jpg")
                if os.path.exists(image_path):
                    images.append(image_path)
                    labels.append(label)
    return images, labels

# Charger les données d'entraînement et de test
train_images, train_labels = load_data(train_file)
test_images, test_labels = load_data(test_file)

# Créer les datasets d'entraînement et de test
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.map(lambda x, y: preprocess_image(x, y)).batch(32).shuffle(1000).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.map(lambda x, y: preprocess_image(x, y)).batch(32).prefetch(tf.data.AUTOTUNE)

# Charger le modèle ResNet50 pré-entraîné
base_model = tf.keras.applications.ResNet50(
    weights='imagenet',  # Charger les poids pré-entraînés sur ImageNet
    include_top=False,  # Ne pas inclure la couche de classification de l'ImageNet
    input_shape=(224, 224, 3)  # Spécifier la forme des images
)

# Congeler les couches du modèle de base pour éviter de les entraîner
base_model.trainable = False

# Ajouter des couches personnalisées pour la classification
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1024, activation='relu'),  # Ajout de la parenthèse fermante manquante
    tf.keras.layers.Dense(num_classes, activation='softmax')  # Classification avec softmax
])

# Compiler le modèle
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Entraîner le modèle
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=10,  # Réduire les epochs pour un entraînement plus rapide
)
